In [30]:
import warnings
warnings.filterwarnings("ignore")

from datasets import load_dataset, concatenate_datasets, DatasetDict
from tensorflow import keras as K
from keras import Input
from keras.models import Sequential 
from keras.layers import Dense, Embedding, LSTM
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Data Preparation

In [31]:
ds = load_dataset('winvoker/turkish-sentiment-analysis-dataset')
ds

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'dataset'],
        num_rows: 440679
    })
    test: Dataset({
        features: ['text', 'label', 'dataset'],
        num_rows: 48965
    })
})

In [32]:
ds = concatenate_datasets([ds['train'], 
                                    ds['test']])

ds = ds.remove_columns('dataset')
ds_combined_reduced = ds.shuffle(seed=42).select(range(len(ds)//10))
del ds


In [33]:
texts = ds_combined_reduced['text']
labels = ds_combined_reduced['label']

In [34]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
seqs = tokenizer.texts_to_sequences(texts)

In [35]:
max_len = max(len(seq) for seq in  seqs)
X = pad_sequences(seqs, maxlen=max_len)

In [36]:
ds_combined_reduced['text'] = X

TypeError: 'Dataset' object does not support item assignment

In [7]:
ds_train = ds_combined_reduced.train_test_split(test_size=.2)
ds_validation = ds_train['test'].train_test_split(test_size=.5)
ds = DatasetDict ({
    'train': ds_train['train'], 
    'validation': ds_validation['train'], 
    'test': ds_validation['test']
})
ds

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 39171
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 4896
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 4897
    })
})

In [8]:
X_train = ds['train']['text']
X_validation = ds['validation']['text']
X_test = ds['test']['text']
y_train = ds['train']['label']
y_validation = ds['validation']['label']
y_test = ds['test']['label']

## Label Encoding

In [23]:
le = LabelEncoder()

y_train_encoded = le.fit_transform(y_train)
y_validation_encoded = le.transform(y_validation)
y_test_encoded = le.transform(y_test)

In [25]:
y_train = K.utils.to_categorical(y_train_encoded, len(le.classes_))
y_validation = K.utils.to_categorical(y_validation_encoded, len(le.classes_))
y_test = K.utils.to_categorical(y_test_encoded, len(le.classes_))

# Model Building

In [29]:
model = Sequential()
model.add(Embedding(max_len, 64))
model.add(LSTM(64, dropout=.3))
model.add(Dense(3, activation='softmax'))

NameError: name 'max_len' is not defined